In [ ]:
from PIL import Image
import torch
!git clone https://github.com/clovaai/donut.git
!cd donut && pip install .
from donut import DonutModel
import json
import shutil

Cloning into 'donut'...
remote: Enumerating objects: 264, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 264 (delta 15), reused 22 (delta 11), pack-reused 229
Receiving objects: 100% (264/264), 62.77 MiB | 12.22 MiB/s, done.
Resolving deltas: 100% (114/114), done.
Checking out files: 100% (66/66), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/donut
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 5.8 MB 5.2 MB/s 
     |████████████████████████████████| 549 kB 44

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("connected to my drive")

Mounted at /content/drive
connected to my drive


In [ ]:
import spacy
import numpy as np
import pandas as pd

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
s = 999
np.random.seed(s)
spacy.util.fix_random_seed(s)


In [ ]:
import spacy
import numpy as np
import pandas as pd

nlp = spacy.load("en_core_web_sm")
text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")
doc = nlp(text)
words = []
labels = []

for token in doc:
	words.append(token.text)
	labels.append('O') # As most of token will be non-entity (OUT). Replace this later with actual entity a/c the scheme.

df = pd.DataFrame({'word': words, 'label': labels})
df.to_csv('ner-token-per-line.biluo', index=False) #

In [ ]:
df

In [ ]:
import spacy
import random
from spacy.gold import GoldParse
from spacy.language import EntityRecognizer

ModuleNotFoundError: ignored

In [ ]:
text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")
g = {'entities': [(5, 20, 'PERSON'), (61, 67, 'ORG'), (71, 75, 'DATE'), (173, 181, 'NORP'),
		  (271, 276, 'PERSON'), (299, 305, 'ORG'), (306, 323, 'DATED')]}

X = [text]
Y = [g]

In [ ]:
X

['When Sebastian Thrun started working on self-driving cars at Google in 2007, few people outside of the company took him seriously. “I can tell you very senior CEOs of major American car companies would shake my hand and turn away because I wasn’t worth talking to,” said Thrun, in an interview with Recode earlier this week.']

In [ ]:
Y

[{'entities': [(5, 20, 'PERSON'),
   (61, 67, 'ORG'),
   (71, 75, 'DATE'),
   (173, 181, 'NORP'),
   (271, 276, 'PERSON'),
   (299, 305, 'ORG'),
   (306, 323, 'DATED')]}]

In [ ]:

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
print(f'[OtherPipes] = {other_pipes} will be disabled')



[OtherPipes] = ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer'] will be disabled


In [ ]:
from spacy.training.example import Example

In [ ]:
add_ents = ['DATED'] # The new entity
# Piplines in core pretrained model are tagger, parser, ner. Create new if blank model is to be trained using `spacy.blank('en')` else get the existing one.
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner") # "architecture": "ensemble" simple_cnn ensemble, bow # https://spacy.io/api/annotation
    nlp.add_pipe(ner)
else:
    ner = nlp.get_pipe("ner")
prev_ents = ner.move_names # All the existing entities recognised by the model
print('[Existing Entities] = ', ner.move_names)
for ent in add_ents:
    ner.add_label(ent)

new_ents = ner.move_names
# print('\n[All Entities] = ', ner.move_names)
print('\n\n[New Entities] = ', list(set(new_ents) - set(prev_ents)))
## Training
model = None # Since we are training a fresh model not a saved model
n_iter = 20
with nlp.disable_pipes(*other_pipes):  # only train ner
    # optimizer = nlp.begin_training()
    if model is None:
        optimizer = nlp.create_optimizer()
    else:
        optimizer = nlp.resume_training()
    for i in range(n_iter):
        losses = {}
        nlp.update(X, Y,  sgd=optimizer, drop=0.0, losses=losses)
        # nlp.entity.update(d, g)
        print("Losses", losses)


[Existing Entities] =  ['B-ORG', 'B-DATE', 'B-PERSON', 'B-GPE', 'B-MONEY', 'B-CARDINAL', 'B-NORP', 'B-PERCENT', 'B-WORK_OF_ART', 'B-LOC', 'B-TIME', 'B-QUANTITY', 'B-FAC', 'B-EVENT', 'B-ORDINAL', 'B-PRODUCT', 'B-LAW', 'B-LANGUAGE', 'I-ORG', 'I-DATE', 'I-PERSON', 'I-GPE', 'I-MONEY', 'I-CARDINAL', 'I-NORP', 'I-PERCENT', 'I-WORK_OF_ART', 'I-LOC', 'I-TIME', 'I-QUANTITY', 'I-FAC', 'I-EVENT', 'I-ORDINAL', 'I-PRODUCT', 'I-LAW', 'I-LANGUAGE', 'L-ORG', 'L-DATE', 'L-PERSON', 'L-GPE', 'L-MONEY', 'L-CARDINAL', 'L-NORP', 'L-PERCENT', 'L-WORK_OF_ART', 'L-LOC', 'L-TIME', 'L-QUANTITY', 'L-FAC', 'L-EVENT', 'L-ORDINAL', 'L-PRODUCT', 'L-LAW', 'L-LANGUAGE', 'U-ORG', 'U-DATE', 'U-PERSON', 'U-GPE', 'U-MONEY', 'U-CARDINAL', 'U-NORP', 'U-PERCENT', 'U-WORK_OF_ART', 'U-LOC', 'U-TIME', 'U-QUANTITY', 'U-FAC', 'U-EVENT', 'U-ORDINAL', 'U-PRODUCT', 'U-LAW', 'U-LANGUAGE', 'O']


[New Entities] =  ['B-DATED', 'U-DATED', 'L-DATED', 'I-DATED']


ValueError: ignored

In [ ]:
import spacy


nlp = spacy.load('en_core_web_sm')

text = "When Sebastian Thrun started working on self-driving cars at"

labels = [(5, 20, 'PERSON')]

extra_stopwords = ['hi']

for word in extra_stopwords:
    nlp.vocab[word].is_stop = True

doc = nlp(text)

# add Span(start, end, label) to doc.ents
for start, end, label in labels:
    span = doc.char_span(start, end, label)

    # I forgot char_span only assigns the label to the span
    # might be safer to use the ._ namespace I mentioned earlier
    # instead of .ent_type_, but the idea should be the same
    for token in span:
        token.ent_type_ = label

# examples of things you can access
print([(token,
        token.ent_type_,
        token.lemma_.lower(),
        token.is_stop)
      for token in doc])

[(When, '', 'when', True), (Sebastian, 'PERSON', 'sebastian', False), (Thrun, 'PERSON', 'thrun', False), (started, '', 'start', False), (working, '', 'work', False), (on, '', 'on', True), (self, '', 'self', False), (-, '', '-', False), (driving, '', 'drive', False), (cars, '', 'car', False), (at, '', 'at', True)]


In [ ]:
from spacy.tokens import Doc


# example
def preprocessing(token):
    """Takes a token and returns the processed string."""
    return token.lemma_.lower()


labels_new = []
words = []
spaces = []
offset = 0


for i, token in enumerate(doc):
    token_new = preprocessing(token)
    words.append(token_new)

    if token.ent_type_:
        labels_new.append((offset,
                           offset + len(token_new),
                           token.ent_type_))

    offset += len(token_new)

    if len(token.whitespace_) == 1:
        spaces.append(True)

    else:
        spaces.append(False)


doc_new = Doc(nlp.vocab, words, spaces)

In [ ]:
print([(token, token.idx) for token in doc_new])

[(when, 0), (sebastian, 5), (thrun, 15), (start, 21), (work, 27), (on, 32), (self, 35), (-, 39), (drive, 40), (car, 46), (at, 50)]


In [ ]:
print(labels_new)

[(4, 13, 'PERSON'), (13, 18, 'PERSON')]


In [ ]:

import spacy


In [ ]:

# !python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_lg


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-08 09:34:47.835749: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 13 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:

nlp = spacy.load("en_core_web_lg")
nlp

In [ ]:

doc = nlp("Aug. 12.   39AM   141 Z 29:48:14 Lincoln Financial - > No.  , P. 2/18 Lincoln Financial Gr Page s Lincoln Financial Group Lincoln Life Assurance Compmy of Boston Group Benefits Disability Claims P.O. Bax 7208  , KY 40742-7208 Phone  .:   ) 213-2937 Secure Fax  .:   ) 214-7839  : August  , 2020  :  . MICHAEL  , MD PENN FAMILY MEDICINE PAH 700 SPRUCE STREET DUNCAN BUILDING STE 305 PHILADELPHIA PA 19106  :  :   ) 829-3477  : Philip Parisio Dis Claims Std Case Mgr II Phone  .: Phone  .:   ) 213-2937 Secure Fax  .:   ) 559-1959 Total Pages  Including   : 3 (  : Claim  : 9846441  : Keith Cringle Wells Fargo & Company This  , and any attachments  , is intended only for the use of the    ) named herein and may contain legally privileged   or confidential  . If you are not the intended recipient of this  , you are hereby notified that any  , distribution or copying of this  , and any attachments  , is strictly  . If you have received this fex in  , plcasc notify me by telephone at   ) 213-2937 and permanently shred the original and any copy of any fax and any printout  . 08/12/2020  AM       ) Aug. 12. 2020  AM 80/12/20 89:48:52 Lincoln Financial  > No. 8782 P. 3/18 Lincoln Financial Gr Page MuÅ¾ Lincoln Financial Group Lincoln Life Assurance Company of Bosto Group Benefits Disability Claims PO Box 7208  , KY 40742-7208 Phone  .:   ) 213-2937 Secure Fax  .:   ) 214-7839 August  , 2020 Dr. Michael  , MD Penn Family Medicine Pah 700 SPRUCE STREET DUNCAN BUILDING STE 305  , PA 19106  :   Term Disability   ) Benefits Wells Fargo & Company Claim  : 9846441  : Keith Cringle Claimant  .: 7/17/1977 Dear Dr. Michael  ,  ; We are the Disability Claim Administrator for your  , Keith Cringle To evaluate eligibility for disability benefits and to help facilitate a return to  , when  , we are requesting the following  : Office treatment  , test  , prescription  , and treatment plans from July  , 2020 through present We ask that you provide this information by August  , 2020. Failure to provide the requested                             faxed           secure fax     -559-1959 or      . If prepayment is  , please fax an invoice indicating the number of  , federal tax  # and amount  . Although HIPAA does not apply to disability insurance  , we understand your responsibilities under HIPAA as a health care  , and our associated responsibility of ensuring this information is protected against deliberate or inadvertent misuse or  . The Genetic Information Nondiscrimination Act of 2008   ) prohibits employers and other entities covered by GINA Title I from requesting or requiring genetic information of an individual or family member of the  , except as specifically allowed by this  . To comply with this  , we are asking that you not provide any genetic information when responding to this request for 1 of 2 08/12/2020  AM       ) Aug. 12. 2020  AM     20 09:49:29 Lincoln Financial No. 8782 P. 4/18 Lincoln Financial Gr Page 883  > medical  .  Genetic  ' as defined by  , includes an individual's family medical  , the results of an individual's or family")

In [ ]:

doc.ents


In [ ]:

from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)